<a href="https://colab.research.google.com/github/ilank-pro/SmartHotel360-Website/blob/master/NDX_Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance pandas plotly


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
import yfinance as yf
import pandas as pd
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Step 1: Retrieve the list of Nasdaq-100 stocks
nasdaq_100_tickers = """
AAPL MSFT AMZN NVDA GOOG GOOGL META TSLA PEP ADBE NFLX CSCO AVGO TXN CMCSA COST INTU SBUX AMGN QCOM AMD BKNG GILD FISV ADP MU MDLZ LRCX ADI HON LLY AMAT ISRG PYPL NOW REGN PANW ORLY TMUS CHTR ABNB ILMN ZM SNPS MRVL FTNT KLAC DXCM NXPI VRTX MS DELL BIIB WDAY EA EXC SEDG MCHP PDD ASML IDXX KDP MAR WMT CDNS ALGN MRNA OKTA PTON DOCU TEAM ZS SPLK MDB DDOG SHOP CRSP NET U DKNG SQ PATH BILL ROKU TWLO RIVN CRWD
"""

# Convert the string of tickers into a list
tickers = nasdaq_100_tickers.split()

# Step 2: Fetch historical stock price data with error handling
valid_tickers = []
data_dict = {}
for ticker in tickers:
    try:
        data = yf.download(ticker, start="2024-01-01", end="2024-05-17")
        if not data.empty:
            valid_tickers.append(ticker)
            data_dict[ticker] = data
    except Exception as e:
        print(f"Failed to download data for {ticker}: {e}")





[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [49]:
# Step 3: Calculate momentum indicators
momentum_stocks = []
for ticker in valid_tickers:
    stock_data = data_dict[ticker]
    if 'Close' not in stock_data.columns:
        continue

    stock_data = stock_data[['Close']].dropna()
    stock_data['Returns'] = stock_data['Close'].pct_change()
    stock_data['Momentum'] = stock_data['Returns'].rolling(window=7).mean()

    # Define momentum criteria
    if stock_data['Momentum'].iloc[-1] > 0.013:
        momentum_stocks.append(ticker)

print(momentum_stocks)

['DELL', 'IDXX', 'MRNA']


[]


In [65]:
# Step 4: Identify breakout stocks
# Function to identify breakout stocks
def identify_breakout_stocks(data, margin=0.005, close_margin=0.003, occurrences=4, window=20):
    breakout_stocks = []
    for ticker, stock_data in data.items():
        highs = stock_data['High']
        closes = stock_data['Close']
        for i in range(len(highs) - window + 1):
            window_highs = highs[i:i + window]
            count = sum((window_highs.max() * (1 - margin) <= window_highs) & (window_highs <= window_highs.max() * (1 + margin)))
            if count > occurrences:
                # Check if the close price in the last 3 days is within 0.3% of the high of the rolling window
                recent_closes = closes[i + window - 3:i + window]
                if any(abs(recent_close - window_highs.max()) / window_highs.max() <= close_margin for recent_close in recent_closes):
                    breakout_stocks.append(ticker)
                    break
    return breakout_stocks

breakout_stocks = identify_breakout_stocks(data_dict)
print(breakout_stocks)

['FISV', 'REGN', 'VRTX', 'EXC', 'MAR', 'WMT', 'SPLK']


In [67]:
# Function to identify special breakout stocks
def identify_special_breakout_stocks(data, close_margin=0.003, rangeInDays=5):
    special_breakout_stocks = []
    for ticker, stock_data in data.items():
        max_high = stock_data['High'].max()
        recent_closes = stock_data['Close'].iloc[-rangeInDays:]
        if all(abs(recent_close - max_high) / max_high <= close_margin for recent_close in recent_closes):
            special_breakout_stocks.append(ticker)
    return special_breakout_stocks

special_breakout_stocks = identify_breakout_stocks(data_dict)
print(special_breakout_stocks)

['FISV', 'REGN', 'VRTX', 'EXC', 'MAR', 'WMT', 'SPLK']


In [66]:
# Step 5: Filter data for the last 3 months and plot candlestick charts for the selected stocks
#for ticker in set(momentum_stocks) | set(breakout_stocks):

for ticker in set(breakout_stocks):
    stock_data = data_dict[ticker][['Open', 'High', 'Low', 'Close']].dropna()
    #stock_data = stock_data[stock_data.index >= three_months_ago]

    fig = make_subplots(rows=1, cols=1)

    fig.add_trace(go.Candlestick(
        x=stock_data.index,
        open=stock_data['Open'],
        high=stock_data['High'],
        low=stock_data['Low'],
        close=stock_data['Close'],
        name=ticker
    ))

    fig.update_layout(
        title=f'Candlestick Chart for {ticker}',
        xaxis_title='Date',
        yaxis_title='Price',
        xaxis_rangeslider_visible=False,
        width=1200,  # Increase the width for bigger candles
        height=600,  # Increase the height for bigger candles
        margin=dict(l=20, r=20, t=40, b=20),
        font=dict(size=14)
    )

    fig.show()